In [1]:
import io
import pyaudio
from google.oauth2 import service_account
from google.cloud import speech

client_file = 'speech_api_keys.json'
credentials = service_account.Credentials.from_service_account_file(client_file)
client = speech.SpeechClient(credentials=credentials)

RATE = 16000
CHUNK = int(RATE / 10)

audio_interface = pyaudio.PyAudio()
stream = audio_interface.open(format=pyaudio.paInt16,
                              channels=1,
                              rate=RATE,
                              input=True,
                              frames_per_buffer=CHUNK)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=RATE,
    language_code='en-US'
)
streaming_config = speech.StreamingRecognitionConfig(config=config, interim_results=True)

def generate_audio_chunks():
    while True:
        yield speech.StreamingRecognizeRequest(audio_content=stream.read(CHUNK))

responses = client.streaming_recognize(config=streaming_config, requests=generate_audio_chunks())

try:
    print("Listening... Press Ctrl+C to stop.")
    for response in responses:
        for result in response.results:
            if result.is_final:
                print("Transcript:", result.alternatives[0].transcript)
except KeyboardInterrupt:
    print("Stopping...")
finally:
    # Clean up
    stream.stop_stream()
    stream.close()
    audio_interface.terminate()

Listening... Press Ctrl+C to stop.
Transcript: to the mic um
Transcript:  hello
Transcript:  oh
Transcript:  hear me the eraser
Transcript:  hand me the eraser
Stopping...
